# Feed Monitoring

This notebook allows monitoring of trades, PnL, win rates & hit ratios.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import ipywidgets as widgets
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly_resampler import FigureResampler, FigureWidgetResampler
import datetime as dt
from typing import Literal
import plotly.io as pio
pio.renderers.default='notebook'
warnings.filterwarnings("ignore")


In [ ]:
# Note that this is meant to work on only 1 type of indices now, i.e DEX/DSI
# This is because of the top_players, which only really makes sense for 1 type of index

# ATM, this uses Metabase, will implement for MT5 downloaded feeds

"""
TODO: Add support for MT5 feeds
"""
def preprocessor(all_indices_paths : list[str], mt5_symbol_names : list[str], mt5_trades_path : str, top_10_winners_path : str, is_ctrader):
    """
    Run this before any following operations
    
    Requires:
     * feed.tick databases
     * MT5 Symbol Names
     * MT5 trades
     * Top 10 Winners

    To prevent unexpected results, ensure all the start/end dates are the same.

    To get the information, e.g for DSI, use the following SQL commands in Metabase:

    feed.tick:
    SELECT underlying, ts, bid, spot, ask
    FROM feed.tick
    WHERE underlying IN ('DSI10') 
    AND   ts >= "2024-01-23"
    AND   ts <= "2024-03-22"
    ORDER BY ts, underlying

    MT5 trades:
    SELECT *
    FROM bi.mt5_deal 
    WHERE time_deal >= "2024-01-23"
    AND   time_deal <= "2024-03-22"
    AND   symbol like ('Drift Switch Index%')
    AND   action in (0,1)
    AND   entry in (0,1,3)

    Top 10 winners:
    Refer to https://metabase-bi.4x.my/question/9226-top-10-winners-drift-switch-index & download the entire df
    """
    assert len(all_indices_paths) == len(mt5_symbol_names), 'Ensure all_indices_path has the same length as mt5_symbol_names'

    all_info = dict()
    date_grouper = pd.Grouper(key='time_deal', freq='D')
    all_trades_raw = pd.read_csv(mt5_trades_path)
    
    if is_ctrader:
        all_trades_raw['action'] = np.where(all_trades_raw.trade_side == 1,0,1)
        all_trades_raw.volume = all_trades_raw.volume*10000
        all_trades_raw.rename(columns={'execution_timestamp' : 'time_deal', 'profit_usd' : 'profit'}, inplace=True)
        all_trades_raw['profit_raw'] = 0 #for now 
        all_trades_raw['commission'] = 0 #for now 
        all_trades_raw['expert_id'] = 0 #for now
        all_trades_raw['price_position'] = np.where((all_trades_raw.profit>=0)|(all_trades_raw.profit<=0),1,0)
        all_trades_raw['entry'] = all_trades_raw['contract_count']
        all_trades_raw['profit'] = np.where((all_trades_raw.price_position==0),0,all_trades_raw.profit)
        
    top_players_raw = pd.read_csv(top_10_winners_path)
    
    # Data massage all trades 
    all_trades = all_trades_raw[['login', 'time_deal', 'action', 'entry', 'price', 'volume', 'profit', 'commission', 'profit_raw', 'price_position', 'symbol','expert_id']]
    
    all_trades.time_deal = pd.to_datetime(all_trades.time_deal, format="mixed")
    
    all_trades.volume = all_trades.volume / 10000
    all_trades = all_trades.sort_values(['login','time_deal'])

    # Saving top player id for use later
    top_player_ids = top_players_raw.Login
    top_player_trades = all_trades[all_trades.login.isin(top_player_ids)]
    top_player_trades = top_player_trades.sort_values(['login','time_deal'])

    # Get player information
    player_info = all_trades[all_trades.price_position !=0]
    player_info['win_lose'] = np.where(player_info.profit>0,1,0)
    player_info['abs_profit'] = abs(player_info.profit)
    player_info['pos_profit'] = np.maximum(player_info.profit, 0)

    # Perform all requisite operations & process columns to get statistics
    player_info = player_info.groupby(['login', 'symbol']).agg({'win_lose' : np.mean, # Win rate
                                                'profit': [np.sum, np.mean], 'volume': [np.sum, np.mean], # Total/Mean profts & volumes
                                                'entry': np.sum, 'expert_id': np.sum, # Total trades (and expert_ids for bot differentiation)
                                                'abs_profit': np.sum, 'pos_profit': np.sum}) # Components of hit rate

    player_info.columns = [' '.join(col).strip() for col in player_info.columns.values]
    player_info.rename(columns={'win_lose mean': 'win_rate', 'profit sum': 'total_PnL', 'profit mean': 'mean_PnL', 'expert_id sum' : 'expert_id',
                        'volume sum': 'total_volume', 'volume mean': 'avg_volume', 'entry sum': 'total_trades'}, inplace=True)

    # Define hit ratios
    player_info['hit_ratio'] = player_info['pos_profit sum'] / player_info['abs_profit sum']
    player_info = player_info.sort_values('total_PnL', ascending=False).reset_index()
    player_info = player_info[['login', 'symbol', 'avg_volume', 'mean_PnL', 'expert_id', 'win_rate', 'total_PnL', 'total_volume', 'total_trades', 'hit_ratio']]

    # Company side daily profits
    company_profit_daily = all_trades.groupby([date_grouper,all_trades.symbol]).sum().reset_index()[['symbol','time_deal','profit','entry','volume']].sort_values(['symbol','time_deal'])
    company_profit_daily.profit = -company_profit_daily.profit

    dates_with_trades = all_trades.time_deal.dt.date.sort_values().unique()

    for index_path, symbol in zip(all_indices_paths, mt5_symbol_names):
        # Pre-process Feeddb feed
        feed = pd.read_csv(index_path, index_col=0)
        feed.rename(columns={'ts': 'date'}, inplace=True)

        # Isolate trades for particular indices
        trades_index = all_trades[all_trades.symbol == symbol].sort_values('time_deal',ascending=True)
        player_info_index = player_info[player_info.symbol == symbol]

        

        # Company side half day profits
        index_firsthalf_pnl = [all_trades[(all_trades.time_deal.dt.date == i) & (all_trades.time_deal.dt.hour < 12)  & (all_trades.symbol == symbol)].profit.sum() for i in dates_with_trades]
        index_secondhalf_pnl = [all_trades[(all_trades.time_deal.dt.date == i) & (all_trades.time_deal.dt.hour >= 12) & (all_trades.symbol == symbol)].profit.sum() for i in dates_with_trades]
        index_halfday_pnl = [None]*(len(index_firsthalf_pnl) + len(index_secondhalf_pnl))
        index_halfday_pnl[::2] = index_firsthalf_pnl
        index_halfday_pnl[1::2] = index_secondhalf_pnl
        index_halfday_pnl = -np.round(index_halfday_pnl,2)

        all_info[symbol] = {'index' : feed,
                            'trades' : trades_index,
                            'halfday_pnl' : index_halfday_pnl,
                            'player_info' : player_info_index
                            }

    return all_trades, player_info, top_player_ids, top_player_trades, dates_with_trades, company_profit_daily, all_info

# Use MT5 symbols
symbols = ['Step Index 300', 'Step Index 400'] 

"""
SELECT underlying, ts, bid, spot, ask
FROM feed.tick
WHERE underlying IN ('DSI10') 
AND   ts >= "2024-01-23"
and   ts <= "2024-03-22"
ORDER BY ts, underlying
"""
all_indices_paths = ['./stpRNG3_demo.csv', './stpRNG4_demo.csv']

"""
SELECT *
FROM bi.mt5_deal 
WHERE time_deal >= "2024-01-23"
AND   time_deal <= "2024-03-22"
AND   symbol like ('Drift Switch Index%')
AND   action in (0,1)
AND   entry in (0,1,3)
"""
mt5_trades_path = './ctrader_demo_trades_details_2024-04-15T02_36_15.471805Z.csv'
is_ctrader = True # True if ctrader trades else please set to False
"""Check corresponding dashboard or look for top 10 winners in Metabase"""
top_10_winners_path = './top_10_winners__c_b_indices_cfd_2024-04-15T05_42_07.534944Z.csv'


# Ensure that the following names are consistent
all_trades, player_info, top_player_ids, top_player_trades, dates, company_profit_daily, all_info = preprocessor(all_indices_paths, symbols, mt5_trades_path, top_10_winners_path, is_ctrader)


In [ ]:
#temporarily hardcoding top player ids 
top_player_ids = pd.Series([2017858, 2032961, 2051310, 2056192, 2028232, 2054615, 2046324, 2019757, 2055130, 2011585])

In [ ]:
valid_client_options = ['All Clients', 'Top Clients', 'EA Clients', 'Not EA Clients']
def plot_dailycum_individual(cumulative, daily, overall, df_length, dates, label, who, port_num):
    """
    Plots an individual figure for daily cumulative graphs.
    """
    fig = FigureResampler(go.Figure())
    # Fit polynomial to cumulative win rates & plot
    x = np.arange(len(cumulative))
    y = cumulative
    fig.add_trace(go.Scatter(name=f'Cumulative {label}', mode='lines', showlegend=True), hf_x=dates, hf_y=cumulative)
    if len(x) == 1:
        coefficients = np.array([y[0],y[0]])
    else:
        coefficients = np.polyfit(x, y, 1)
    polynomial = np.poly1d(coefficients)
    fig.add_trace(go.Scatter(
        x=dates,
        y=polynomial(x),
        mode='lines',
        name='Cumulative Trendline',
        line=dict(color='red')
    ))

    # Fit polynomial to daily win rates & plot
    x = np.arange(len(daily))
    y = daily
    fig.add_trace(go.Scatter(line_color='teal',name=f'Daily {label}', mode='lines', showlegend=True), hf_x=dates, hf_y=daily)
    if len(x) == 1:
        coefficients = np.array([y[0],y[0]])
    else:
        coefficients = np.polyfit(x, y, 1)    
    polynomial = np.poly1d(coefficients)
    fig.add_trace(go.Scatter(
    x=dates,
    y=polynomial(x),
    mode='lines',
    name='Daily Trendline',
    line=dict(color='tan')
    ))
    annotations = [
        go.layout.Annotation(x=0.05, y=-0.2, xref='paper', yref='paper', text=f"Total Trades: {df_length}    Overall {label}: {overall}", showarrow=False,
                                font=dict(color='black', size=15))
    ]
    fig.update_layout(
        xaxis=dict(title='Date'),
        yaxis=dict(title=label),
        title=f'Cumulative Daily {label} ({who})',
        showlegend= True,
        annotations=annotations
    )
    print(port_num)
    fig.show_dash(mode='inline',port = port_num)

def plot_dailycum_all(show_who):
    """
    Plots daily cumulative win rates and hit ratios of a particular category of client.

    Includes:
    Top Clients: Rates of the top 10 clients
    EA Clients: Clients using Experts
    Not EA Clients: Clients not using Experts
    All Clients: All of the above
    """
    # For ensuring port numbers are distinct
    port_num = 8010
    start = pd.to_datetime(all_info[symbols[0]]['index'].date.iloc[0]).tz_localize('UTC')
    end = pd.to_datetime(all_info[symbols[0]]['index'].date.iloc[-1]).tz_localize('UTC')
    start1 = np.datetime64(start)
    end1 = np.datetime64(end)
    
    if show_who == 'All':
        to_process = valid_client_options
    elif show_who in valid_client_options:
        to_process = [show_who]
    else: 
        raise ValueError(f'Must be one of the following: {valid_client_options}')
    port_num = 8060
    for who in to_process:
        port_num += 2
        if who == 'All Clients':
            closed_trades_df = all_trades[all_trades.price_position!=0].sort_values('time_deal')
        elif who == 'Top Clients':
            closed_trades_df = top_player_trades[top_player_trades.price_position!=0].sort_values('time_deal')
        elif who == 'EA Clients':
            closed_trades_df = all_trades[(all_trades.price_position!=0)&(all_trades.expert_id!=0)].sort_values('time_deal')
        elif who == 'Not EA Clients':
            closed_trades_df = all_trades[(all_trades.price_position!=0)&(all_trades.expert_id==0)].sort_values('time_deal')
            
        if len(closed_trades_df!=0):
            dates = closed_trades_df.time_deal.dt.date.sort_values().unique()
            closed_trades_df = closed_trades_df[(closed_trades_df.time_deal.dt.tz_localize('UTC') >= start) & (closed_trades_df.time_deal.dt.tz_localize('UTC') <= end)]
            closed_trades_df.reset_index(drop=True,inplace=True)

            # Get win rates (cumulative & daily)
            closed_trades_df['win_lose']=np.where(closed_trades_df.profit>0,1,0)
            win_rates_cumulative = [closed_trades_df[closed_trades_df.time_deal.dt.date<=i].win_lose.mean() for i in dates]
            win_rates_daily = [closed_trades_df[closed_trades_df.time_deal.dt.date==i].win_lose.mean() for i in dates]
            win_rate_overall = round(len(closed_trades_df[closed_trades_df.profit>0])/len(closed_trades_df),5)

            # Calculate hit ratios
            closed_trades_df['profit_abs']=np.abs(closed_trades_df.profit)
            closed_trades_df['profit_pos']=np.maximum(closed_trades_df.profit,0)
            hit_ratios_daily = [closed_trades_df[closed_trades_df.time_deal.dt.date==i].profit_pos.sum() / closed_trades_df[closed_trades_df.time_deal.dt.date==i].profit_abs.sum() for i in dates]
            hit_ratios_cumulative = [closed_trades_df[closed_trades_df.time_deal.dt.date<=i].profit_pos.sum() / closed_trades_df[closed_trades_df.time_deal.dt.date<=i].profit_abs.sum() for i in dates]
            hit_ratios_overall = round(closed_trades_df.profit_pos.sum()/closed_trades_df.profit_abs.sum(),5)

            # Plot daily/cumulative
            plot_dailycum_individual(win_rates_cumulative, win_rates_daily, win_rate_overall, len(closed_trades_df), dates, 'Win Rate', who, port_num)
            plot_dailycum_individual(hit_ratios_cumulative, hit_ratios_daily, hit_ratios_overall, len(closed_trades_df), dates, 'Hit Ratio', who, port_num+1)

show_who = widgets.RadioButtons(
    options=valid_client_options+['All'],
    value='All',
    description='Clients:',
    disabled=False
)

plot_dailycum_all_ui = widgets.HBox([show_who])
plot_dailycum_all_out = widgets.interactive_output(plot_dailycum_all, {'show_who':show_who})

In [ ]:
def plot_dailycum(login_ids : str | int | list[str | int], EA = 'All'):
    '''
    If set EA, can plot for EA only/filter over EA for the selected
    '''
    port_num = 8020
    start = pd.to_datetime(all_info[symbols[0]]['index'].date.iloc[0]).tz_localize('UTC')
    end = pd.to_datetime(all_info[symbols[0]]['index'].date.iloc[-1]).tz_localize('UTC')
    start1 = np.datetime64(start)
    end1 = np.datetime64(end)
    
    if isinstance(login_ids, (str, int)):
        id_flag = (all_trades.login == int(login_ids)) 
    elif isinstance(login_ids, (list, np.ndarray, pd.Series, pd.DataFrame)):
        id_flag = (all_trades.login.isin(login_ids))

    if EA == 'All':
        EA_flag = (all_trades.expert_id == all_trades.expert_id)
    elif EA == 'EA': # We have EA trades if expert_df > 0
        EA_flag = (all_trades.expert_id!=0)
    elif EA == 'Not EA':
        EA_flag = (all_trades.expert_id==0)

    closed_trades_df = all_trades[(all_trades.price_position!=0) & id_flag & EA_flag].sort_values('time_deal')    
    dates = closed_trades_df.time_deal.dt.date.sort_values().unique()
    closed_trades_df = closed_trades_df[(closed_trades_df.time_deal.dt.tz_localize('UTC') >= start ) & (closed_trades_df.time_deal.dt.tz_localize('UTC') <= end)]
    closed_trades_df.reset_index(drop=True,inplace=True)
    if len(closed_trades_df)!=0:
    # Get win rates (cumulative & daily)
        closed_trades_df['win_lose']=np.where(closed_trades_df.profit>0,1,0)
        win_rates_cumulative = [closed_trades_df[closed_trades_df.time_deal.dt.date<=i].win_lose.mean() for i in dates]
        win_rates_daily = [closed_trades_df[closed_trades_df.time_deal.dt.date==i].win_lose.mean() for i in dates]
        win_rate_overall = round(len(closed_trades_df[closed_trades_df.profit>0])/len(closed_trades_df),5)

        # Calculate hit ratios
        closed_trades_df['profit_abs']=np.abs(closed_trades_df.profit)
        closed_trades_df['profit_pos']=np.maximum(closed_trades_df.profit,0)
        hit_ratios_daily = [closed_trades_df[closed_trades_df.time_deal.dt.date==i].profit_pos.sum() / closed_trades_df[closed_trades_df.time_deal.dt.date==i].profit_abs.sum() for i in dates]
        hit_ratios_cumulative = [closed_trades_df[closed_trades_df.time_deal.dt.date<=i].profit_pos.sum() / closed_trades_df[closed_trades_df.time_deal.dt.date<=i].profit_abs.sum() for i in dates]
        hit_ratios_overall = round(closed_trades_df.profit_pos.sum()/closed_trades_df.profit_abs.sum(),5)

        if login_ids is None and EA is None:
            who = 'All Clients'

        who = str(login_ids) + ' ' + ('' if EA is None else EA)
        # Plot daily/cumulative
        plot_dailycum_individual(win_rates_cumulative, win_rates_daily, win_rate_overall, len(closed_trades_df), dates, 'Win Rate', who, 8020)
        plot_dailycum_individual(hit_ratios_cumulative, hit_ratios_daily, hit_ratios_overall, len(closed_trades_df), dates, 'Hit Ratio', who, 8021)

top_player_selection_widget = widgets.Dropdown(
    options=top_player_ids,
    value=top_player_ids.iloc[0],
    description='Top player id:',
    disabled=False
)

player_login_widget = widgets.IntText(
    value=top_player_ids.iloc[0],
    description='Player id:',
    disabled=False
)

ea_widget = widgets.RadioButtons(
    options=['All', 'EA', 'Not EA'],
    value='All',
    description='Trades:',
    disabled=False
)

plot_dailycum_player_ui = widgets.HBox([player_login_widget, ea_widget])
plot_dailycum_top_player_ui = widgets.HBox([top_player_selection_widget, ea_widget])

plot_dailycum_player_out = widgets.interactive_output(plot_dailycum, {'login_ids':player_login_widget, 'EA': ea_widget})
plot_dailycum_top_player_out = widgets.interactive_output(plot_dailycum, {'login_ids':top_player_selection_widget, 'EA': ea_widget})

In [ ]:
def plot_pnl ():
    port_num = 8030
    for symbol, info_dict in all_info.items():
        index = info_dict['index']
        hday_pnl = info_dict['halfday_pnl']
        
        fig = FigureResampler(go.Figure())
        fig.add_trace(
            go.Scatter(name='Spot', mode='lines', showlegend=True),
            hf_x=index.date, 
            hf_y=index.spot
        )
        for i in company_profit_daily.time_deal:
            fig.add_shape(type='rect',
                        xref='x', yref='paper',
                        x0=dt.datetime.combine(i,dt.time(0, 0, 0)), y0=0,
                        x1=dt.datetime.combine(i,dt.time(23, 59, 59)), y1=1,
                        fillcolor='green' if company_profit_daily[(company_profit_daily.time_deal==i) * (company_profit_daily.symbol == symbol)].profit.values[0]>0 else 'red',
                        opacity=0.15,
                        layer='below')

        annotations = [
            go.layout.Annotation(x=0.1, y=-0.15, xref='paper', yref='paper', text='Win', showarrow=False,
                                font=dict(color='green', size=12)),
            go.layout.Annotation(x=0.3, y=-0.15, xref='paper', yref='paper', text='Lose', showarrow=False,
                                font=dict(color='red', size=12))
        ]
        fig.update_layout(
            xaxis=dict(title='Time'),
            yaxis=dict(title='Spot'),
            title=f'Company Win Rate by Day for {symbol}',
            showlegend= True,
            annotations=annotations
        )
        fig.show_dash(mode='inline',port = port_num)
        port_num+=1
        fig = FigureResampler(go.Figure())
        fig.add_trace(
            go.Scatter(name='Spot', mode='lines', showlegend=True),
            hf_x=index.date, 
            hf_y=index.spot
        )
        for i in range(len(dates)):
            fig.add_shape(type='rect',
                        xref='x', yref='paper',
                        x0=dt.datetime.combine(dates[i],dt.time(0, 0, 0)), y0=0,
                        x1=dt.datetime.combine(dates[i],dt.time(11, 59, 59)), y1=1,
                        fillcolor='green' if hday_pnl[i*2]>0 else 'red',
                        opacity=0.3,
                        layer='below')
            fig.add_shape(type='rect',
                        xref='x', yref='paper',
                        x0=dt.datetime.combine(dates[i],dt.time(12, 0, 0)), y0=0,
                        x1=dt.datetime.combine(dates[i],dt.time(23, 59, 59)), y1=1,
                        fillcolor='green' if hday_pnl[i*2+1]>0 else 'red',
                        opacity=0.3,
                        layer='below')
        annotations = [
            go.layout.Annotation(x=0.1, y=-0.15, xref='paper', yref='paper', text='Win', showarrow=False,
                                font=dict(color='green', size=12)),
            go.layout.Annotation(x=0.3, y=-0.15, xref='paper', yref='paper', text='Lose', showarrow=False,
                                font=dict(color='red', size=12))
        ]
        fig.update_layout(
            xaxis=dict(title='Time'),
            yaxis=dict(title='Spot'),
            title=f'Company Win Rate by 12 Hours for {symbol}',
            showlegend= True,
            annotations=annotations
        )
        fig.show_dash(mode='inline',port = port_num)
        port_num+=1


In [ ]:
def plot_trades(s,e, show_ma, login_id = 0,):#, trade_selection):
    start = pd.to_datetime(np.datetime64(s)).tz_localize('UTC')
    end = pd.to_datetime(np.datetime64(e)).tz_localize('UTC')
    start1 = np.datetime64(s)
    end1 = np.datetime64(e)
    port_num = 8050
    for symbol, info_dict in all_info.items():
        index = info_dict['index']
        dates = pd.to_datetime(index.date)
        trades_all = info_dict['trades']
        login_flag = trades_all.login.isin([login_id])
                                                
        spot = index[(dates >= start1) & (dates <= end1)]
        trades = trades_all[(trades_all.time_deal.dt.tz_localize('UTC') >= start) & (trades_all.time_deal.dt.tz_localize('UTC') <= end) & login_flag].reset_index(drop=True)
        print('Number of Trades', len(trades[trades.profit != 0]))
        print('Total PnL', round(trades[trades.price_position !=0].profit.sum(),2))
        print('Mean PnL', round(trades[trades.price_position !=0].profit.mean(),2))

        fig = FigureResampler(go.Figure())
        fig.add_trace(go.Scatter(name='spot', mode='lines', showlegend=True),hf_x=spot.date, hf_y=spot.spot)
        if show_ma:
            fig.add_trace(go.Scatter(line_color = 'orange',name='Moving Average', mode='lines', showlegend=True),hf_x=spot.date, hf_y=spot.spot.rolling(window=show_ma).mean(), )
        
        actions = trades['action'].to_numpy()
        time_deals = trades['time_deal'].to_numpy()
        min_spot = min(spot.spot)
        max_spot = max(spot.spot)

        shapes = []
        for i in trades.index:
            if actions[i] == 0:
                color = 'green'
            else: 
                color='red'
            shapes.append({'type': 'line',
                        'xref': 'x',
                        'yref': 'y',
                        'x0': time_deals[i],
                        'y0': min_spot,
                        'x1': time_deals[i],
                        'y1': max_spot,
                        'line': dict(color=color)})

        annotations = [
            go.layout.Annotation(x=0.1, y=-0.15, xref='paper', yref='paper', text='buy', showarrow=False,
                                font=dict(color='green', size=12)),
            go.layout.Annotation(x=0.3, y=-0.15, xref='paper', yref='paper', text='sell', showarrow=False,
                                font=dict(color='red', size=12))
        ]
        fig.update_layout(
            xaxis=dict(title='Time'),
            yaxis=dict(title='Spot'),
            title=f'Trades on {symbol} by ' + str(login_id),
            showlegend= True,
            annotations=annotations,
            shapes=shapes
        )
        fig.show_dash(mode='inline',port = port_num)
        port_num+=1
        
start_date = widgets.Text(
    value=all_info[symbols[0]]['index'].date.iloc[0],
    placeholder='Type Start Date',
    description='Start Date:',
    disabled=False
)
end_date = widgets.Text(
    value=all_info[symbols[0]]['index'].date.iloc[-1],
    placeholder='Type End Date',
    description='End Date:',
    disabled=False
)
show_ma = widgets.IntText(
    value=0,
    description='Moving Average:',
    disabled=False
)
ids= widgets.IntText(
    value=top_player_ids[0],
    description='ID:',
    disabled=False
)
top_player_id= widgets.Dropdown(
    value=top_player_ids[0],
    options=top_player_ids,
    description='ID:',
    disabled=False
)
plot_trades_ui = widgets.HBox([start_date, end_date, ids, show_ma])
plot_trades_out = widgets.interactive_output(plot_trades, {'s': start_date,'e': end_date,'login_id':ids,'show_ma':show_ma})

plot_top_player_trades_ui = widgets.HBox([start_date, end_date, top_player_id, show_ma])
plot_top_player_trades_out = widgets.interactive_output(plot_trades, {'s': start_date,'e': end_date,'login_id':top_player_id,'show_ma':show_ma})

In [ ]:
# Top players
# display(plot_dailycum_top_player_ui,plot_dailycum_top_player_out)

# Individual player (Manual)
display(plot_dailycum_player_ui,plot_dailycum_player_out)

# Multiple players (pre-defined list)
# players = top_player_ids[:5]
# plot_dailycum(players, 'All')

In [ ]:
def show_trades(trade_selection : str, show_opens, login_id ):
    if trade_selection == 'None':
        print()
        return
    elif trade_selection == 'All':
        trades = all_trades
    else:
        trades = all_info[trade_selection]['trades']
        
    if show_opens:
        display(trades[(trades.time_deal >= start_date.value) & (trades.time_deal <= end_date.value) & (trades.login == login_id)].sort_values(['symbol','time_deal']))
    else:
        display(trades[(trades.price_position != 0) & (trades.time_deal >= start_date.value) & (trades.time_deal <= end_date.value) & (trades.login == login_id)].sort_values(['symbol','time_deal']))

trade_selection = widgets.RadioButtons(
    options=['All'] + symbols + ['None'],
    value='None',
    description='Show Trades For:',
    disabled=False
)
show_opens = widgets.Checkbox(
    value=False,
    description='Opens',
    disabled=False
)

show_trades_id = widgets.IntText(
    value=top_player_ids.iloc[0],
    description='Player id:',
    disabled=False
)

show_trades_ui = widgets.HBox([trade_selection,show_opens,show_trades_id])
show_trades_out = widgets.interactive_output(show_trades, {'trade_selection': trade_selection,'show_opens':show_opens, 'login_id':show_trades_id})

## Plots

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

### Cumulative Daily Win Rates & Hit Ratios

Displays cumulative daily win rates & hit ratios for an individual or a list of individuals.

#### Top Players
1. Select Top player with drop-down menu
2. Select types of trades from EA (bot), Not EA or All trades

#### Individual
1. Type Player Login ID in textbox
2. Select types of trades from EA (bot), Not EA or All trades

#### Multiple
1. Define list of login ids to observe
2. Select types of trades from EA (bot), Not EA or All trades (`'EA', 'Not EA', 'All`)

In [ ]:
# Top players
# display(plot_dailycum_top_player_ui,plot_dailycum_top_player_out)

# Individual player (Manual)
display(plot_dailycum_player_ui,plot_dailycum_player_out)

# Multiple players (pre-defined list)
# players = top_player_ids[:5]
# plot_dailycum(players, 'All')

### Overall Cumulative Daily Win Rates & Hit Ratios

Displays cumulative daily win rates & hit ratios for all clients in a category

1. Select category of clients from
   * All Clients
   * Top Clients
   * EA Clients
   * Not EA Clients
   * All of the above

In [ ]:
display(plot_dailycum_all_ui,plot_dailycum_all_out)

### Overall Company Win Rates

In [ ]:
plot_pnl()

### Top Player Trades

Displays feed & trades for top players.

1. Select dataframes of trades to show
2. Filter to only opened trades
3. Select start & end dates
4. Choose top player login id
5. Choose moving average duration (OPTIONAL)

In [ ]:
display(show_trades_ui,show_trades_out)
display(plot_top_player_trades_ui,plot_top_player_trades_out)